This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [121]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [122]:
# First, import the relevant modules
import requests
import json


In [123]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
payload = {'start_date':'2017-10-02', 'end_date':'2017-10-02',
           'api_key': API_KEY}
resp = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json', params=payload)


In [124]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
data = resp.json()
data


{'dataset': {'id': 10095370,
  'dataset_code': 'AFX_X',
  'database_code': 'FSE',
  'name': 'Carl Zeiss Meditec (AFX_X)',
  'description': 'Stock Prices for Carl Zeiss Meditec (AFX) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704',
  'refreshed_at': '2018-11-23T22:34:22.231Z',
  'newest_available_date': '2018-11-23',
  'oldest_available_date': '2000-06-07',
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'frequency': 'daily',
  'type': 'Time Series',
  'premium': False,
  'limit': None,
  'transform': None,
  'column_index': None,
  'start_date': '2017-10-02',
  'end_date': '2017-10-02',
  'data': [['2017-10-02',
    44.51,
    44.98,
    44.18,
    44.98,
    None,
    95313.0,
    4265024.0,
    None,
    None,
    None]],
  'collapse': None,
  'order': None,
  'database_id': 6129}}

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [125]:
#Task 1: Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017
#Select all data between Jan 1st, 2017 and Dec 31st 2017
payload = {'start_date': '2017-01-01', 'end_date': '2017-12-31',
           'api_key': API_KEY}
resp = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json', params=payload)


In [126]:
#Task 2: Convert the returned JSON object into a Python dictionary.
#Call on json library and check data type
data = resp.json()
type(data)

dict

In [127]:
#Task 3: Calculate the highest and lowest opening prices in this period.
#Create lists using comprehensions from the 3rd and 4th elements of data which should correspond 
#to each day's high and low then call max and min functions to find the highest and lowest values
highest = max([list[2] for list in data['dataset']['data']])
lowest = min([list[3] for list in data['dataset']['data']])
highest, lowest

(53.54, 33.62)

In [128]:
#Task 4: What was the largest change in any one day (based on High and Low price)?
#Use similar strategy as the one in task 3, but subtract the low from the high price to get change
largest_change = max([list[2] - list[3] for list in data['dataset']['data']])
largest_change

2.8100000000000023

In [129]:
#Task 5: What was the largest change between any two days (based on Closing Price)?
#Create two lists of closing prices: one as they are in the data and another offset by one day
#Subtracing the corresponding elements of the first list from the second list will give a change value for each day
t0 = [list[4] for list in data['dataset']['data']]
t1 = t0[1:]
#Create a list of changes in closing prices calculated by subtracting the previous day's closing price
change = [y - x for x,y in zip(t0,t1)]
#Write a function to report the biggest change that accounts for both increases and decreases in closing price
def maxchange(x):
    if abs(min(x)) > abs(max(x)):
        print(min(x))
    else:
        print(max(x))
        
#Call function on change list
maxchange(change)

2.559999999999995


In [130]:
#Task 6: What was the average daily trading volume during this year?
#Create list of trading volume using list comprehension and generate average 
#Note: I tried to do this without importing any package - a more parsimonious way would have involved calling a mean function
#from a package like NumPy or statistics
volume = [list[6] for list in data['dataset']['data']]
average_volume = float(sum(volume))/float(len(volume))
average_volume

89124.33725490196

In [131]:
#Task 7: What was the median trading volume during this year. 
#Create list of daily volume and sort with ascending as default

volume = [list[6] for list in data['dataset']['data']]
volume.sort()

#Write function for median and apply it to sorted volume

def median(x):
    value1 = x[int(len(x)/2)]
    value2 = x[int((len(x)/2)+1)]
    if len(x)%2 > 0:
        print(x[int(len(x)/2)])
    else:
        print((value1+value2)/2)
        
median(volume)



76286.0
